In [1]:
# !pip install catboost
# !pip install xgboost

# 1. Import Necessary Libraries 

In [2]:
# import necessary libraries
import pandas as pd #library for data manipulation
import numpy as np # library for working with arrayS

# creating visualizations in Python
import matplotlib.pyplot as plt
import plotly.express as px
from pylab import rcParams
import seaborn as sns
sns.set_style('darkgrid')
rcParams['figure.figsize'] = 8,8
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

# import modelling libraries
from catboost import CatBoostRegressor #features built in support for handling categorical features
from xgboost import XGBRegressor #designed to be fast and efficient: for both classification and regression
from sklearn.ensemble import RandomForestRegressor #creates multiple decision trees
from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold, RepeatedStratifiedKFold

# pd.options.display.max_rows=2000
pd.set_option('display.max_rows', 100) #sets maximum rows to 100
pd.set_option('display.max_columns', 30) #sets maximum columns to 30

# 2. Read Data

In [3]:
# read csv data
df = pd.read_csv('food_prices_kenya.csv', parse_dates=True, index_col='date')

In [4]:
# check sample data/records
df.sample(10)

,admin1,admin2,market,latitude,longitude,category,commodity,unit,priceflag,pricetype,currency,price,usdprice
date,,,,,,,,,,,,,
11/15/2021,Nyanza,Kisumu,Kibuye (Kisumu),-0.0927162,34.7691,cereals and tubers,Sorghum (red),90 KG,actual,Wholesale,KES,3996,35.9838
6/15/2021,Rift Valley,Turkana,Kalobeyei (Village 3),3.713947283,34.85746868,miscellaneous food,Sugar,KG,aggregate,Retail,KES,120,1.1241
11/15/2021,Rift Valley,Baringo,Marigat town (Baringo),0.469569,35.980663,cereals and tubers,Millet (finger),90 KG,actual,Wholesale,KES,7200,64.8357
10/15/2021,Rift Valley,Turkana,Kalobeyei (Village 2),3.760887785,34.75283168,pulses and nuts,Beans (dry),KG,aggregate,Retail,KES,106.67,0.9706
5/15/2021,North Eastern,Garissa,IFO (Daadab),0.110807,40.316079,"meat, fish and eggs",Meat (beef),KG,aggregate,Retail,KES,480,4.4559
1/15/2015,Nyanza,Kisumu,Kisumu,-0.1,34.75,cereals and tubers,Potatoes (Irish),50 KG,actual,Wholesale,KES,1483.33,16.5413
1/15/2021,Nairobi,Nairobi,Mathare (Nairobi),-1.256,36.8572,"meat, fish and eggs",Meat (goat),KG,aggregate,Retail,KES,520,4.7532
3/15/2021,Rift Valley,Turkana,Mogadishu (Kakuma),3.71708,34.850333,pulses and nuts,Beans (dry),KG,aggregate,Retail,KES,115,1.0642
10/15/2018,Coast,Mombasa,Mombasa,-4.05,39.666667,pulses and nuts,Beans (dry),90 KG,actual,Wholesale,KES,5243.75,52.5163


# 3. Explore Data

Data Type Description and Statistical Information

In [5]:
# check shape of the dataset
df.shape

(14755, 13)

In [6]:
# check information of the dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14755 entries, #date to 8/15/2022
Data columns (total 13 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   admin1     14755 non-null  object
 1   admin2     14755 non-null  object
 2   market     14755 non-null  object
 3   latitude   14755 non-null  object
 4   longitude  14755 non-null  object
 5   category   14755 non-null  object
 6   commodity  14755 non-null  object
 7   unit       14755 non-null  object
 8   priceflag  14755 non-null  object
 9   pricetype  14755 non-null  object
 10  currency   14755 non-null  object
 11  price      14755 non-null  object
 12  usdprice   14755 non-null  object
dtypes: object(13)
memory usage: 1.6+ MB


### 
* From the above, all columns are categorical. Shows there are anomalies since some 
  columns cant have object as a datatype i.e price etc.
* No missing values from the dataset

In [7]:
# check statistical information of numerical data
df.describe()

,admin1,admin2,market,latitude,longitude,category,commodity,unit,priceflag,pricetype,currency,price,usdprice
count,14755,14755,14755,14755,14755,14755,14755,14755,14755,14755,14755,14755,14755
unique,8,23,63,63,63,9,48,15,4,3,2,6474,10547
top,Rift Valley,Nairobi,Nairobi,-1.283333,36.816667,cereals and tubers,Maize (white),KG,actual,Wholesale,KES,50,0
freq,5183,3460,1835,1835,1835,7262,1869,6161,10283,8260,14754,301,286


# 4. Data Preprocessing

In [8]:
# check for duplicates
df.duplicated().sum()

251

In [9]:
# remove duplicates
df = df.drop_duplicates()
df.duplicated().sum()

0

In [10]:
# check for missing values and sorting the issue
missing_perc = df.isnull().mean()*100
missing_values = pd.DataFrame({'column_name' : df.columns, 'Missing Percentage' : missing_perc})
missing_values

,column_name,Missing Percentage
admin1,admin1,0.0
admin2,admin2,0.0
market,market,0.0
latitude,latitude,0.0
longitude,longitude,0.0
category,category,0.0
commodity,commodity,0.0
unit,unit,0.0
priceflag,priceflag,0.0
pricetype,pricetype,0.0


### 
- No missing values from the dataset

In [11]:
# make a copy of the dataframe
df1 = df.copy()

In [12]:
# rename some columns
df1 = df1.rename(columns={'admin1': 'region', 'admin2': 'county'})

In [13]:
# column names
print(*df1.columns, sep='\n')

region
county
market
latitude
longitude
category
commodity
unit
priceflag
pricetype
currency
price
usdprice


In [14]:
# check datatypes of the above columns
for col in df1:
    print(f'{col} : {df1[col].dtypes}')

region : object
county : object
market : object
latitude : object
longitude : object
category : object
commodity : object
unit : object
priceflag : object
pricetype : object
currency : object
price : object
usdprice : object


In [15]:
# remove some patterns in the columns you want to convert into numerical
import re  # Import the regular expression module

# Define the numeric conversion function
def numeric(text):
    if isinstance(text, str):
        text = text.lower()
        if re.search(r'#', text):
            text = '0'  # Replace '#' with '0'
    return text

# Apply the numeric function to selected columns in the DataFrame
df1[num_cols] = df1[num_cols].applymap(numeric)


NameError: name 'num_cols' is not defined

In [ ]:
# convert necessary columns to their respective datatypes
num_cols = ['latitude', 'longitude', 'price', 'usdprice']
for num in num_cols:
    df1[num] = df1[num].astype('float64')
    
df1.dtypes

### 
- I now have a clean dataset with their corresponding data types

In [ ]:
# check statistical information of numerical data
df1.describe()

### 
- No outliers in my dataset

### 4.1 Quality mapping

In [ ]:
# columns
df1.columns

In [ ]:
# check category 
print(*df1['category'].unique(), sep='\n')

### 
- category seems to be ok

In [ ]:
# check commodity 
print(*df1['commodity'].unique(), sep='\n')

In [ ]:
# do a quality mapping on commodity column
import re
def commodity(text):
    text = text.lower()
    if re.match(r'^ma', text):
        text = 'Maize'
    if re.match(r'^b', text):
        text = 'Beans'
    if re.match(r'^p', text):
        text = 'Potatoes'
    if re.match(r'^mi', text):
        text = 'Milk'
    if re.match(r'^fu', text):
        text = 'Fuel'
    if re.match(r'^mea', text):
        text = 'Meat'
    if re.match(r'^oi', text):
        text = 'Oil'
    if re.match(r'^on', text):
        text = 'Onion'
    if re.match(r'^ri', text):
        text = 'Rice'
    if re.match(r'^so', text):
        text = 'Sorghum'
    if re.match(r'^cow', text):
        text = 'Cowpeas'
    if re.match(r'^mil', text):
        text = 'Millet'
    if re.match(r'^fis', text):
        text = 'Fish'
    return text
        
df1['commodity'] = df1['commodity'].apply(commodity)
print('Unique values in Commodity column: ')
print('-------------------------------')
print(*df1['commodity'].unique(), sep='\n')
print('-------------------------------')

In [ ]:
# check unit column 
print(*df1['unit'].unique(), sep='\n')

In [ ]:
# confirm unique values of our columns

# list of column names to exclude from the analysis
excluded_cols = ['region', 'county', 'market', 'price', 'usdprice', 'latitude', 'longitude'] 

# loop through each column in the DataFrame
for col in df1.columns:
    # check if the column is not in the list of excluded columns
    if col not in excluded_cols:
        # retrieve unique values from the column and convert them to strings
        unique_values = "\n".join(df1[col].unique().astype(str))
        # print column name, number of unique values, and the unique values themselves
        print(f'{col} : {df1[col].nunique()} : {unique_values}\n')
        print('------------------------------------------')

In [ ]:
df1.dtypes

# 5. Feature Engineering
- Here we will pick our essential features for training the model

In [ ]:
# let us see our columns
df1.columns

In [ ]:
# create a copy of the df1 column
df2 = df1.copy()

In [ ]:
# drop unnecessary columns
df2.drop(columns=['usdprice', 'currency'], axis=1, inplace=True)

In [ ]:
# drop more unnecessary columns
df2.drop(columns=['latitude', 'longitude'], axis=1, inplace=True)

In [ ]:
# new columns to fit our model with
df2.columns

In [ ]:
df2.dtypes

In [ ]:
df2.to_csv('df_preprocessed.csv', index=False)
df2 = pd.read_csv ('df_preprocessed.csv')

# 6. Model Selection and Training

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Specify the chosen categorical columns to encode
cat_cols = ['region', 'county', 'market' , 'category', 'commodity', 'unit', 'priceflag', 'pricetype']

# Create an instance of LabelEncoder
label_encoder = LabelEncoder()

# Encode the categorical columns
for col in cat_cols:
    df2[col] = label_encoder.fit_transform(df2[col].astype(str))

# Print the encoded DataFrame
print(df2)


In [ ]:
df2.head(10)

In [ ]:
df2.tail()

In [ ]:
df2.columns

In [ ]:
df3 = df2.copy()

In [ ]:
df3.head()

In [ ]:
df3.columns

In [ ]:
df3.shape

In [ ]:
# split data (70%-30%) into train set and test set 
from sklearn.model_selection import train_test_split

X = df3.drop('price', axis=1)
y = df3['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

print ('Training Set: %d rows\nTest Set: %d rows' % (X_train.shape[0], X_test.shape[0]))

In [ ]:
X_train.columns

In [ ]:
# Initialize the model
from sklearn.metrics import r2_score
cat = CatBoostRegressor(loss_function='RMSE', n_estimators=100, learning_rate=0.05, max_depth=5)
xgb = XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.05, max_depth=5)
linear = LinearRegression()
rf = RandomForestRegressor()
models= [cat,linear, rf,xgb]

# Fit the model to the training data
for model in models:
    model = model.fit(X_train, y_train)
    
    # Make predictions on the test data
    y_pred = model.predict(X_test)
    
    #metric
    mse = r2_score(y_test, y_pred) * 100
    print(f'error for {model} is: {mse}')
    


# Comparisons between the trained models
- From the trained models above: RandomForestRegressor, XGBRegressor and CatBoostRegressor are doing fairly well.
- However, linear regression would be the best to use in this case as it has the lowest mse.

In [ ]:
import joblib

# Save the model
joblib.dump(linear, 'linear.joblib')


In [ ]:
# View trained model
import joblib
from sklearn.linear_model import LinearRegression

# Load the trained model
model = joblib.load('linear.joblib')

# Print the model attributes
print(model.coef_)  # coefficients of the linear regression model
print(model.intercept_)  # intercept of the linear regression model
print(model.get_params())  # parameters used to train the model


In [ ]:
import joblib

# Save the model
joblib.dump(cat, 'catboost.joblib')